In [ ]:
import pandas as pd

from aust_covid.inputs import get_ifrs
from emutools.tex import StandardTexDoc
from aust_covid.inputs import get_base_vacc_data
from aust_covid.vaccination import add_derived_data_to_vacc, get_model_vacc_vals_from_data
from aust_covid.model import build_model
from aust_covid.plotting import plot_immune_props
from inputs.constants import PROJECT_PATH, SUPPLEMENT_PATH
from emutools.inputs import load_param_info

In [ ]:
vacc_df = get_base_vacc_data()
ext_vacc_df = add_derived_data_to_vacc(vacc_df)
boost_data = get_model_vacc_vals_from_data(ext_vacc_df, 'prop boosted in preceding')
primary_data = get_model_vacc_vals_from_data(ext_vacc_df, 'prop primary full in preceding')

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
param_info = load_param_info(PROJECT_PATH / 'inputs' / 'parameters.yml')
param_info['value'].update(get_ifrs(app_doc))
parameters = param_info['value'].to_dict()
epi_model = build_model(app_doc)
epoch = epi_model.get_epoch()

In [ ]:
model_comps = [c.name for c in epi_model._original_compartment_names]
for patch in epi_model.stratifications['states'].strata:
    for age in epi_model.stratifications['agegroup'].strata:
        sub_strat = f'{patch}_{age}'
        epi_model.request_output_for_compartments(sub_strat, model_comps, {'agegroup': age, 'states': patch})

In [ ]:
epi_model.run(parameters=parameters)

In [ ]:
derived_outs = epi_model.get_derived_outputs_df()
start_pops_df = pd.DataFrame()
for patch in ['wa', 'other']:
    patch_data = derived_outs[[i for i in derived_outs.columns if f'{patch}_' in i]].iloc[0, :]
    patch_data.index = patch_data.index.str.replace(f'{patch}_', '')
    start_pops_df[patch] = patch_data
start_pops_df.plot()

In [ ]:
plot_immune_props(epi_model, ext_vacc_df)